In [1]:
from dotenv import load_dotenv
import os

env_file = '.env'
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')

else:
    print(f"File {env_file} not found.")

In [2]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(HOST, auth=(USERNAME, PASSWORD))

In [3]:
import os

os.makedirs("source-csvs", exist_ok=True)

In [ ]:
%pip install --upgrade google-cloud-bigquery

In [10]:
!gcloud auth application-default login > /dev/null 2>&1

In [11]:
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client()

bq_data_id = "neo4jeventdemos.bomv1"

In [13]:
# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(bq_data_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, exists_ok=True, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset neo4jeventdemos.bomv1


In [14]:
# Suppliers
res_df = driver.execute_query("""
MATCH (s:Supplier)
OPTIONAL MATCH (s)-[:LOCATED_AT]->(l)
    RETURN s.code AS code,
        s.ANNUAL_SPEND AS annual_spend,
        s.type AS tier,
        s.sub_type AS sub_type,
        s.category AS category,
        l.latitude AS latitude,
        l.longitude AS longitude
""", result_transformer_= lambda r: r.to_df())
res_df.to_csv('source-csvs/suppliers.csv', index=False)
res_df

,code,annual_spend,tier,sub_type,category,latitude,longitude
0,1BYKA5,0.00,Tier1,Manu,None,-71.854680,71.681478
1,0Q6W9M,0.00,Tier1,Direct,None,-71.854680,71.681478
2,U5TLHT,8056092.85,Tier1,Direct,None,-71.854680,71.681478
3,1QZ9ME,0.00,Tier1,Direct,None,-71.854680,71.681478
4,WH3ZVG,4801.61,Tier1,Direct,None,-71.629455,70.897790
...,...,...,...,...,...,...,...
13885,07XTEF,0.00,Tier1,Manu,None,-68.723473,66.523889
13886,P3OZZV,98107.83,Tier1,Direct,None,-68.723473,66.523889
13887,7PM532,643743.20,Tier1,Direct,None,-71.930702,71.005756
13888,ENEDNC,5453.94,Tier1,Direct,None,-71.536288,71.746616


In [15]:
job_config = bigquery.LoadJobConfig(
    # BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)
table_id = f"{bq_data_id}.suppliers"
job = client.load_table_from_dataframe(
    res_df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=neo4jeventdemos, location=US, id=368b3984-eb92-4699-9fd8-cfb842cfa13a>

In [16]:
# SupplierItems
print("Reading Data...")
res = driver.execute_query("""
MATCH (i:Product|Component)-[:AT]->(s:Supplier)
    RETURN s.code AS supplier_code,
        i.skuId AS sku_id
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/supplier-items.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.supplier_items"
job = client.load_table_from_dataframe(
    res, table_id, job_config=job_config
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,supplier_code,sku_id
0,1BYKA5,28044675
1,1BYKA5,28025519
2,U5TLHT,28612413
3,U5TLHT,28131814
4,U5TLHT,28712502
...,...,...
482406,7PM532,28450613
482407,ENEDNC,9397882
482408,ENEDNC,9397882
482409,KVP039,35158616


In [19]:
# Items
print("Reading Data...")
res = driver.execute_query("""
MATCH (i:Product|Component)
OPTIONAL MATCH (i)-[:GROUP_OF]->(g)
RETURN i.skuId AS sku_id,
    g.code AS group_code,
    CASE labels(i)
        WHEN ['Product'] THEN 'PRODUCT'
        WHEN ['Component'] THEN 'COMPONENT'
        ELSE 'PRODUCT_AND_COMPONENT'
    END AS type,
    i.MANUFACTURER AS manufacturer,
    i.LIFECYCLE_STATUS AS lifecycle_status,
    i.WAFER_PROCESS_NAME AS wafers_process_name,
    i.MPN AS mpn,
    i.WAFER_GEOMETRY AS wafers_geometry,
    i.PACKAGE_METHOD AS package_method,
    i.WAFER_DIAMETER AS wafers_diameter
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/items.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.items"
item_job_conf = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("group_code", bigquery.enums.SqlTypeNames.INTEGER),
    ],
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    res, table_id, job_config=item_job_conf
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,sku_id,group_code,type,manufacturer,lifecycle_status,wafers_process_name,mpn,wafers_geometry,package_method,wafers_diameter
0,12243661,1000.0,COMPONENT,None,None,None,None,None,None,None
1,28499719,1001.0,COMPONENT,None,None,None,None,None,None,None
2,28246025,1001.0,COMPONENT,None,None,None,None,None,None,None
3,28722634,1001.0,COMPONENT,None,None,None,None,None,None,None
4,28263736,1001.0,COMPONENT,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
233604,25367709,NaN,PRODUCT,None,None,None,None,None,None,None
233605,25372957,NaN,PRODUCT,None,None,None,None,None,None,None
233606,28052594,NaN,PRODUCT,None,None,None,None,None,None,None
233607,28092207,NaN,PRODUCT,None,None,None,None,None,None,None


In [20]:
# BOMTable
print("Reading Data...")
res = driver.execute_query("""
MATCH (parent:Product|Component)-[:BOM]->(child)
RETURN parent.skuId AS parent_sku_id,
    child.skuId AS child_sku_id
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/bom-table.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.bom_table"
job = client.load_table_from_dataframe(
    res, table_id, job_config=job_config
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,parent_sku_id,child_sku_id
0,28722634,28824371-01
1,28722634,28824362-01
2,28722634,28755526-01
3,28722634,28824361-01
4,28722634,28809116-01
...,...,...
7478471,33116186,35119186
7478472,13651156,35715222
7478473,13651156,35715220
7478474,13651156,35715221


In [21]:
# ComponentGroups
print("Reading Data...")
res = driver.execute_query("""
MATCH path=(g:Group)-[:SUB_CAT]->(s)-[:CAT_OF]->(c)
WHERE NOT (s.code STARTS WITH "DL" AND c.code = "AW") //throw out 2 odd cases
WITH g.code AS code, nodes(collect(path)[0]) AS path_nodes //minority of multi-subcategories...just picking first one
RETURN code, path_nodes[1].code AS sub_category,  path_nodes[2].code AS category
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/component-groups.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.component_groups"
group_job_conf = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("code", bigquery.enums.SqlTypeNames.INTEGER),
    ],
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    res, table_id, job_config=group_job_conf
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,code,sub_category,category
0,1205,AW-0012,AW
1,1212,AW-0010,AW
2,1262,AW-0010,AW
3,1150,AW-0010,AW
4,1243,AW-0010,AW
...,...,...,...
312,1103,HU-0013,HU
313,1299,HU-0001,HU
314,1296,HU-0000,HU
315,1163,HU-0004,HU


In [22]:
# CustomerFamilies
print("Reading Data...")
res = driver.execute_query("""
MATCH (n:CustomerFamily)
RETURN n.code AS code, n.ANNUAL_REVENUE AS annual_revenue
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/customer-families.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.customer_families"
job = client.load_table_from_dataframe(
    res, table_id, job_config=job_config
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,code,annual_revenue
0,ABJ6813,11022043.75
1,ABO6097,1633.86
2,ABT1245,160543.79
3,ABU5981,61425.00
4,ABW1783,8045.16
...,...,...
1159,GBW7820,0.00
1160,GCI2943,0.00
1161,GCJ9703,0.00
1162,GCL7169,0.00


In [23]:
# Customers
print("Reading Data...")
res = driver.execute_query("""
MATCH (c:Customer)
OPTIONAL MATCH (c)-[:LOCATED_AT]->(l)
OPTIONAL MATCH (c)-[:PART_OF]->(customerFamily)
    RETURN c.code AS code,
        customerFamily.code AS cust_family_code,
        c.ANNUAL_REVENUE AS annual_revenue,
        l.latitude AS latitude,
        l.longitude AS longitude
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/customers.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.customers"
customer_job_conf = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("code", bigquery.enums.SqlTypeNames.INTEGER),
    ],
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    res, table_id, job_config=customer_job_conf
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,code,cust_family_code,annual_revenue,latitude,longitude
0,241,FGK5621,62893.20,85.204849,115.270783
1,276,EVM6571,626261.59,85.338050,115.387250
2,293,FGK5621,22475.60,85.120131,116.282726
3,433,FRO5891,50755.95,28.575001,-2.084160
4,445,FRO5891,72740.80,28.575001,-2.084160
...,...,...,...,...,...
5442,554363,BOR3941,3690230.40,26.231283,85.414119
5443,554388,FPV1408,1436645.42,26.231283,85.414119
5444,554452,FGK5621,16817.76,26.231283,85.414119
5445,554523,FLZ4789,8340556.19,30.661761,91.304522


In [24]:
# CustomerItems
print("Reading Data...")
res = driver.execute_query("""
MATCH (i:Product|Component)-[:AT]->(c:Customer)
    RETURN c.code AS customer_code,
        i.skuId AS sku_id
""", result_transformer_= lambda r: r.to_df())
print("Writing CSV...")
res.to_csv('source-csvs/customer-items.csv', index=False)
print("Writing to BigQuery...")
table_id = f"{bq_data_id}.customer_items"
customer_item_job_conf = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("customer_code", bigquery.enums.SqlTypeNames.INTEGER),
    ],
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    res, table_id, job_config=customer_item_job_conf
)
job.result()
res

Reading Data...
Writing CSV...
Writing to BigQuery...


/Users/zachblumenfeld/interpeters/graphrag-examples/lib/python3.9/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


,customer_code,sku_id
0,241,10769880
1,241,10713430
2,241,10916001
3,241,DEC85280353
4,241,10769876
...,...,...
635805,554523,12191818
635806,555066,10756526
635807,555066,10865162
635808,555066,10756526
